In [1]:
#general imports
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import scipy.optimize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
import random
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

#fmclp imports
from fmclp import fmclp, cuae, synthetic_dataset, benefit, write_res,general_results_write,run_experiment

In [2]:
loan = pd.read_csv('loan_cleaned.csv')
loan = loan[loan['loan_status'] != 'Current']

#create risk groups 0 - good, 1 - bad, 2  - dubious
def loan_grouper(x):
    if x == 'Fully Paid':
        z = 0
    elif x == 'Charged Off':
        z = 1
    elif x == 'Late (31-120 days)':
        z = 2
    elif x == 'Issued':
        z = 2
    elif x=='In Grace Period':
        z= 2
    elif x == 'Late (16-30 days)':
        z =2
    elif x == 'Does not meet the credit policy. Status:Fully Paid':
        z=2
    elif x == 'Default':
        z = 1
    elif x == 'Does not meet the credit policy. Status:Charged Off':
        z = 1
    return z
loan['target'] = loan['loan_status'].apply(loan_grouper)
del loan['loan_status']
del loan['zip_code']
del loan['issue_d']
del loan['addr_state']

loan = loan[['loan_amnt', 'term', 'int_rate', 'verification_status','initial_list_status','target', 'sub_grade',
                'home_ownership', 'purpose', 'dti', 'revol_bal','total_pymnt', 'total_rec_prncp']]

In [3]:
loan = pd.get_dummies(loan, drop_first = True)
loan = loan.rename(columns = {'initial_list_status_w': 'attr'})

In [4]:
y = loan.drop('target', axis=1)
x = loan['target']
y_train,y_test,x_train,x_test = train_test_split(y,x)

lg = LGBMClassifier()
lg.fit(y_train, x_train)
lg = lg.predict(y_test)
accuracy_score(lg, x_test)

0.9380952380952381

In [4]:
experiment = run_experiment(dataset = loan,
                   number_experiments = 100,
                   multiplier = 20,
                   number_iterations = 20,
                   interior_classifier = 'rf',
                   folder = 'loan',
                   dataset_name='loan',
                   initial_classifier = LGBMClassifier())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [6]:
fair_downgraded= np.array( 
[0.04046596, 0.0364651  ,0.03541157, 0.03668359 ,0.04001139, 0.03563529,
 0.03835216, 0.03723847 ,0.03402066, 0.03587653 ,0.03960957, 0.0380548,
 0.03886676, 0.03947823 ,0.0382751 , 0.03841065 ,0.03902686, 0.03784917,
 0.03668067 ,0.03728489 ,0.03911147, 0.03905537 ,0.04066326, 0.03883475,
 0.03911737, 0.03725411 ,0.03540526, 0.03644185, 0.03511794, 0.03920295,
 0.0378252  ,0.03585965 ,0.03562419, 0.03600996, 0.03817524, 0.03591071,
 0.03515498 ,0.03823905 ,0.03966132, 0.03577565, 0.03761201, 0.03494514,
 0.03884597 ,0.03843889, 0.03899055, 0.03823506, 0.03910365, 0.0400073,
 0.03892677 ,0.03600996, 0.03853951, 0.03668288, 0.03893989, 0.03925184,
 0.03844905 ,0.0359811 , 0.03735952, 0.03814118, 0.03752845, 0.03784453,
 0.03622909 ,0.03785898, 0.03978403, 0.03959072, 0.03706946, 0.03814975,
 0.03768334 ,0.03892677, 0.03906774, 0.03584841, 0.03841065, 0.03769894,
 0.03867806 ,0.03974021, 0.03504054, 0.03903527, 0.03657275, 0.03822987,
 0.03820416 ,0.03715378, 0.04007008, 0.03206926, 0.03960957, 0.0418633,
 0.03762798 ,0.03626365, 0.04047454, 0.03503727, 0.03974021, 0.03390552,
 0.03662915 ,0.03806895, 0.03688458, 0.0418633,  0.03923556, 0.03789392,
 0.03878405 ,0.03971383, 0.03781882, 0.03979028])

In [9]:
unfair_downgraded=np.array(
[0.04950818, 0.04989854 ,0.04993888, 0.05012133, 0.05074615 ,0.04819693,
 0.05015438, 0.05010451 ,0.049272   ,0.04960655, 0.05052448 ,0.05018704,
 0.05006977, 0.04998273 ,0.0490204  ,0.05003359, 0.05015712 ,0.04963759,
 0.04961127, 0.0490216  ,0.05071508 ,0.04984885, 0.04911619 ,0.04908648,
 0.05030246, 0.04990874 ,0.04937191 ,0.04979304, 0.04901921 ,0.04975447,
 0.04971545, 0.04906789 ,0.05143229 ,0.04878246, 0.0498495  ,0.04901326,
 0.04898978, 0.04938196 ,0.05136931 ,0.04862134, 0.04752809 ,0.04863876,
 0.0502808 , 0.04901821 ,0.04970179 ,0.04964669, 0.05012506 ,0.0507746,
 0.04982546, 0.04878246 ,0.04930463 ,0.04917499, 0.0502575  ,0.05039718,
 0.04944609, 0.04932763 ,0.04997469 ,0.04906471, 0.04955705 ,0.04912081,
 0.0489407 , 0.04965445 ,0.04987212 ,0.05047108, 0.05011807 ,0.04916898,
 0.05033925, 0.04982546 ,0.04950072 ,0.04991576, 0.05003359 ,0.05125018,
 0.04902596, 0.05104526 ,0.0495582  ,0.04971625, 0.04933163 ,0.04997262,
 0.04905301, 0.04966039 ,0.04954267 ,0.04786002, 0.05052448 ,0.05008137,
 0.05011657, 0.04912943 ,0.04983517 ,0.04986228, 0.05104526 ,0.04884178,
 0.04928506, 0.0508736  ,0.04901046 ,0.05008137, 0.0496727  ,0.05153084,
 0.04953586, 0.05001829 ,0.0500092  ,0.05291822])

In [10]:
fair_downgraded.mean(), unfair_downgraded.mean()

(0.0378428729, 0.049751082199999984)